In [1]:
import pathlib
import subprocess
import pandas as pd
import glob
import os

In [1]:
#group_name = 'group_0'

In [6]:
allc_table_path = f'/data/tmp/Multi.TE.beds/{group_name}.tsv'
output_path = f'{group_name}.mcds'
chrom_size_path = f'/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes'
cpu = 95

In [7]:
pathlib.Path('tmpallc').mkdir(exist_ok = True)

In [8]:
tag_path = f'{group_name}.copy.finish'
tag_path = pathlib.Path(tag_path)

if tag_path.exists() == False:
    tmpdf = pd.read_csv(allc_table_path, sep = '\t', header = None)
    paths = tmpdf[1].tolist()
    
    with open('gcs_paths.txt', 'w') as f:
        for path in paths:
            # Replace the prefix and write the GCS path to the file
            gcs_path = path.replace('/data', 'gs://ecker-rachel-amb-datasets')
            gcs_tbi_path = f"{gcs_path}.tbi"
            f.write(gcs_path + '\n')
            f.write(gcs_tbi_path + '\n')
    
    command = "gsutil -m cp -I tmpallc < gcs_paths.txt"
    subprocess.run(command, shell=True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)
    
else:
    print(f"{group_name} already copied")

In [9]:
subprocess.run(f'touch {group_name}.copy.finish', shell=True)

CompletedProcess(args='touch group_0.copy.finish', returncode=0)

In [10]:
te_to_path = {}
for path in glob.glob(f"/ref/repeatmasker-with-id/*.bed"):
    te_name = pathlib.Path(path).name.split('.')[-2].split('-')[-1]
    te_to_path[te_name] = path

In [11]:
allc_table = pd.read_csv(allc_table_path, sep = '\t', header = None)
allc_table[1] = allc_table[1].apply(lambda x:pathlib.Path(x).name)
curretn_allc_dir = f"{ os.getcwd()}/tmpallc"
allc_table[1] = curretn_allc_dir + '/' + allc_table[1]
allc_table

,0,1
0,AMB_220405_8wk_5E_6E_1_P1-1-K15-A1,/home/qzeng_salk_edu/project/240319_te/tmpallc...
1,AMB_220405_8wk_5E_6E_1_P1-1-K15-A13,/home/qzeng_salk_edu/project/240319_te/tmpallc...
2,AMB_220405_8wk_5E_6E_1_P1-1-K15-A14,/home/qzeng_salk_edu/project/240319_te/tmpallc...
3,AMB_220405_8wk_5E_6E_1_P1-1-K15-A2,/home/qzeng_salk_edu/project/240319_te/tmpallc...
4,AMB_220405_8wk_5E_6E_1_P1-1-K15-B1,/home/qzeng_salk_edu/project/240319_te/tmpallc...
...,...,...
595,AMB_220405_8wk_5E_6E_1_P5-4-C23-K7,/home/qzeng_salk_edu/project/240319_te/tmpallc...
596,AMB_220405_8wk_5E_6E_1_P5-4-C23-K8,/home/qzeng_salk_edu/project/240319_te/tmpallc...
597,AMB_220405_8wk_5E_6E_1_P5-4-C23-L19,/home/qzeng_salk_edu/project/240319_te/tmpallc...
598,AMB_220405_8wk_5E_6E_1_P5-4-C23-L20,/home/qzeng_salk_edu/project/240319_te/tmpallc...


In [12]:
allc_table.to_csv('use_allc_table.csv', sep = '\t', index = None, header = None)

In [13]:
allc_table_path = 'use_allc_table.csv'

In [14]:
command = f"allcools generate-dataset  \
--allc_table {allc_table_path} \
--output_path {output_path} \
--chrom_size_path {chrom_size_path} \
--obs_dim cell \
--cpu {cpu} \
--chunk_size 30 \
--regions DNA {te_to_path['DNA']} \
--quantifiers DNA count CGN,CHN \
--regions LINE {te_to_path['LINE']} \
--quantifiers LINE count CGN,CHN \
--regions LTR {te_to_path['LTR']} \
--quantifiers LTR count CGN,CHN \
--regions LowComplexity {te_to_path['LowComplexity']} \
--quantifiers LowComplexity count CGN,CHN \
--regions RC {te_to_path['RC']} \
--quantifiers RC count CGN,CHN \
--regions RNA {te_to_path['RNA']} \
--quantifiers RNA count CGN,CHN \
--regions SINE {te_to_path['SINE']} \
--quantifiers SINE count CGN,CHN \
--regions Satellite {te_to_path['Satellite']} \
--quantifiers Satellite count CGN,CHN \
--regions SimpleRepeat {te_to_path['SimpleRepeat']} \
--quantifiers SimpleRepeat count CGN,CHN \
--regions rRNA {te_to_path['rRNA']} \
--quantifiers rRNA count CGN,CHN \
--regions scRNA {te_to_path['scRNA']} \
--quantifiers scRNA count CGN,CHN \
--regions snRNA {te_to_path['snRNA']} \
--quantifiers snRNA count CGN,CHN \
--regions srpRNA {te_to_path['srpRNA']} \
--quantifiers srpRNA count CGN,CHN \
--regions tRNA {te_to_path['tRNA']} \
--quantifiers tRNA count CGN,CHN "

In [15]:
tag_path = f'/data/female-amb/Famale.Multi.TE.zarr/{group_name}.finish'
tag_path = pathlib.Path(tag_path)

In [ ]:
if tag_path.exists() == False:
    subprocess.run(f'{command}', shell = True)
    subprocess.run(f'gsutil -m cp -r {group_name}.mcds gs://ecker-rachel-amb-datasets/female-amb/Famale.Multi.TE.zarr', 
                   shell = True, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)  
else:
    print(f'{group_name} done')

command	generate-dataset
allc_table	use_allc_table.csv
output_path	group_0.mcds
chrom_size_path	/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes
obs_dim	cell
cpu	95
chunk_size	30
regions	[['DNA', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-DNA.bed'], ['LINE', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-LINE.bed'], ['LTR', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-LTR.bed'], ['LowComplexity', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-LowComplexity.bed'], ['RC', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-RC.bed'], ['RNA', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-RNA.bed'], ['SINE', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-SINE.bed'], ['Satellite', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-Satellite.bed'], ['SimpleRepeat', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-SimpleRepeat.bed'], ['rRNA', '/ref/repeatmasker-with-id/hg38.repeatmasker.repClass-rRNA.bed'], ['scRNA', '/ref/repeatmasker-wit

In [14]:
subprocess.run(f'touch {group_name}.finish', shell = True)
subprocess.run(f'gsutil cp {group_name}.finish gs://ecker-rachel-amb-datasets/female-amb/Famale.Multi.TE.zarr', 
                   shell = True)

Copying file://test.finish [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


CompletedProcess(args='gsutil cp test.finish gs://ecker-rachel-amb-datasets/female-amb/Famale.Multi.TE.zarr', returncode=0)

In [15]:
subprocess.run(f'rm {group_name}.copy.finish', shell = True)
subprocess.run(f'rm tmpallc -rf', shell = True)

CompletedProcess(args='rm tmpallc -rf', returncode=0)

In [20]:
# command = f"-allc_table {allc_table} \
# --output_prefix {group_name} \
# --dtype uint32 \
# --chrom_size_path  {chrom_size_path} \
# --mc_contexts CGN CHN \
# --cpu 45 \
# --region_bed_paths hg38.repeatmasker.repClass-DNA.bed hg38.repeatmasker.repClass-LINE.bed hg38.repeatmasker.repClass-SINE.bed hg38.repeatmasker.repClass-LTR.bed hg38.repeatmasker.repClass-RC.bed hg38.repeatmasker.repClass-Retroposon.bed hg38.repeatmasker.repClass-RNA.bed hg38.repeatmasker.repClass-rRNA.bed hg38.repeatmasker.repClass-Satellite.bed hg38.repeatmasker.repClass-scRNA.bed hg38.repeatmasker.repClass-snRNA.bed hg38.repeatmasker.repClass-srpRNA.bed hg38.repeatmasker.repClass-tRNA.bed hg38.repeatmasker.repClass-LowComplexity.bed hg38.repeatmasker.repClass-SimpleRepeat.bed \ 
# --region_bed_names DNA LINE SINE LTR RC Retroposon RNA rRNA Satellite scRNA snRNA srpRNA tRNA LowComplexity SimpleRepeat \
# --engine netcdf \
# --max_per_mcds 400"